<div style="padding:10px; color:white; margin:5; font-size:170%; text-align:left; display:fill; border-radius:15px; background-color:#294B8E; overflow:hidden; font-weight:700">
Multimodal Waste Classifier (Image + Short Text) — CLIP-based
</div>

In [1]:
!pip install -q transformers==4.43.0 accelerate -q
!pip install -q scikit-learn -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 71.0 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 75.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.9 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 28.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
import random
from pathlib import Path
from typing import List, Tuple

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, TensorDataset
import torchvision.transforms as T
from PIL import Image
import numpy as np

from transformers import CLIPProcessor, CLIPModel
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix

# Hyperparameters (edit)
IMGSZ = 224               # CLIP-vit-base expects 224
BATCH = 32                # for preprocessing embedding pass
CLS_BATCH = 64            # for classifier training
EMBED_DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
CLASSIFIER_DEVICE = EMBED_DEVICE
SEED = 42
TRAIN_VAL_SPLIT = 0.8

In [3]:
# Paths
INPUT_ROOT = Path("/kaggle/input/garbage-classification/Garbage classification")
OUT_DIR = Path("/kaggle/working/multimodal_clip")
OUT_DIR.mkdir(parents=True, exist_ok=True)
EMB_DIR = OUT_DIR / "embeddings"
EMB_DIR.mkdir(exist_ok=True)

random.seed(SEED)
torch.manual_seed(SEED)

In [4]:
# dataset root detection (handles inner folder)
img_root = None
if INPUT_ROOT.exists():
    candidate = INPUT_ROOT / "Garbage classification"
    if candidate.exists():
        img_root = candidate
    else:
        img_root = INPUT_ROOT
else:
    raise FileNotFoundError(f"Dataset {INPUT_ROOT} not found")

print("Using image root:", img_root)

Using image root: /kaggle/input/garbage-classification/Garbage classification/Garbage classification


In [5]:
# build file list and labels; use provided split files if present else random split
def read_list_file(path: Path) -> List[str]:
    if not path.exists():
        return []
    return [ln.strip() for ln in path.read_text().splitlines() if ln.strip()]


all_images = []
class_names = sorted([p.name for p in img_root.iterdir() if p.is_dir()])
if not class_names:
    raise RuntimeError("No class subfolders found under image root.")

class_to_idx = {c:i for i,c in enumerate(class_names)}

for cls in class_names:
    folder = img_root / cls
    for ext in ("*.jpg","*.png","*.jpeg","*.JPG","*.PNG"):
        for p in folder.glob(ext):
            all_images.append((str(p), class_to_idx[cls], cls))

print(f"Found {len(all_images)} images across {len(class_names)} classes: {class_names}")

# If you have custom text descriptions per image (CSV or JSON), you can load them here.
# For now we auto-generate short text descriptions:
def make_text_for_class(cls_name: str) -> str:
    return f"a photo of a {cls_name}"

# Split into train/val
random.shuffle(all_images)
n_train = int(len(all_images) * TRAIN_VAL_SPLIT)
train_list = all_images[:n_train]
val_list   = all_images[n_train:]
print("Train size:", len(train_list), "Val size:", len(val_list))

Found 2527 images across 6 classes: ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']
Train size: 2021 Val size: 506


In [8]:
#  load CLIP, helper to compute embeddings for a list dataset
device = EMBED_DEVICE
print("Loading CLIP on", device)
model_name = "openai/clip-vit-base-patch32"   # good default
processor = CLIPProcessor.from_pretrained(model_name)
clip = CLIPModel.from_pretrained(model_name).to(device)
clip.eval()  # we freeze it

# Helper dataset for DataLoader to feed PIL images + text
# Helper dataset for DataLoader to feed PIL images + text
class ImageTextListDataset(Dataset):
    def __init__(self, items):
        # items: list of (img_path, class_idx, cls_name)
        self.items = items
    def __len__(self):
        return len(self.items)
    def __getitem__(self, idx):
        path, label, clsname = self.items[idx]
        img = Image.open(path).convert("RGB")
        text = make_text_for_class(clsname)   # or use a custom per-image text
        return img, text, label, path


# ---- Custom collate_fn ----
def collate_fn(batch):
    imgs, texts, labs, pths = zip(*batch)  # unzip
    proc = processor(images=list(imgs), text=list(texts), return_tensors="pt", padding=True)
    return proc["pixel_values"], proc["input_ids"], proc["attention_mask"], torch.tensor(labs), list(pths)


# ---- Compute embeddings ----
def compute_embeddings(items, batch_size=BATCH, split_name="train"):
    ds = ImageTextListDataset(items)
    dl = DataLoader(ds, batch_size=batch_size, shuffle=False, num_workers=2, collate_fn=collate_fn)

    img_embs, txt_embs, labels, paths = [], [], [], []
    with torch.no_grad():
        for pixel_values, input_ids, attention_mask, labs, pths in dl:
            pixel_values = pixel_values.to(device)
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)

            # compute embeddings
            image_features = clip.get_image_features(pixel_values=pixel_values)  # (B, dim)
            text_features = clip.get_text_features(input_ids=input_ids, attention_mask=attention_mask)  # (B, dim)

            # normalize (CLIP typically normalizes)
            image_features = image_features / image_features.norm(p=2, dim=-1, keepdim=True)
            text_features = text_features / text_features.norm(p=2, dim=-1, keepdim=True)

            img_embs.append(image_features.cpu())
            txt_embs.append(text_features.cpu())
            labels.append(labs)
            paths.extend(pths)

    img_embs = torch.cat(img_embs, dim=0)
    txt_embs = torch.cat(txt_embs, dim=0)
    labels = torch.cat(labels, dim=0)

    # save
    torch.save(
        {"img_emb": img_embs, "txt_emb": txt_embs, "labels": labels, "paths": paths},
        EMB_DIR / f"{split_name}_emb.pt"
    )
    print(f"Saved embeddings to {EMB_DIR / f'{split_name}_emb.pt'}")
    return img_embs, txt_embs, labels, paths

Loading CLIP on cuda


In [9]:
# compute and save embeddings 
train_img_emb, train_txt_emb, train_labels, train_paths = compute_embeddings(train_list, batch_size=BATCH, split_name="train")
val_img_emb, val_txt_emb, val_labels, val_paths = compute_embeddings(val_list, batch_size=BATCH, split_name="val")
print("Train embeddings shape:", train_img_emb.shape, train_txt_emb.shape)
print("Val embeddings shape:", val_img_emb.shape, val_txt_emb.shape)

Saved embeddings to /kaggle/working/multimodal_clip/embeddings/train_emb.pt
Saved embeddings to /kaggle/working/multimodal_clip/embeddings/val_emb.pt
Train embeddings shape: torch.Size([2021, 512]) torch.Size([2021, 512])
Val embeddings shape: torch.Size([506, 512]) torch.Size([506, 512])


In [10]:
#  build TensorDataset and dataloaders for classifier
from torch.utils.data import TensorDataset

def make_concat_dataset(img_emb, txt_emb, labels):
    # embeddings are normalized; we will concat them
    X = torch.cat([img_emb, txt_emb], dim=1)  # shape (N, dim_img + dim_txt)
    y = labels
    return TensorDataset(X, y)

train_ds = make_concat_dataset(train_img_emb, train_txt_emb, train_labels)
val_ds   = make_concat_dataset(val_img_emb, val_txt_emb, val_labels)

train_dl = DataLoader(train_ds, batch_size=CLS_BATCH, shuffle=True, num_workers=2)
val_dl   = DataLoader(val_ds, batch_size=CLS_BATCH, shuffle=False, num_workers=2)

print("Classifier input dimension:", train_ds.tensors[0].shape[1])

Classifier input dimension: 1024


In [11]:
# define classifier MLP and training loop
class MultimodalClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim=512, num_classes= len(class_names), dropout=0.3):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(True),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, num_classes)
        )
    def forward(self, x):
        return self.net(x)

input_dim = train_ds.tensors[0].shape[1]
model = MultimodalClassifier(input_dim, hidden_dim=512, num_classes=len(class_names)).to(CLASSIFIER_DEVICE)

# training setup
epochs = 20
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

def evaluate(model, dl):
    model.eval()
    preds = []
    truths = []
    with torch.no_grad():
        for X, y in dl:
            X = X.to(CLASSIFIER_DEVICE)
            logits = model(X)
            p = torch.argmax(logits, dim=1).cpu().numpy()
            preds.extend(p.tolist())
            truths.extend(y.numpy().tolist())
    acc = accuracy_score(truths, preds)
    f1 = f1_score(truths, preds, average="weighted")
    return acc, f1, preds, truths

best_val = 0.0
for epoch in range(1, epochs+1):
    model.train()
    total_loss = 0.0
    for X, y in train_dl:
        X = X.to(CLASSIFIER_DEVICE)
        y = y.to(CLASSIFIER_DEVICE)
        optimizer.zero_grad()
        logits = model(X)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()
        total_loss += float(loss.item()) * X.size(0)
    train_loss = total_loss / len(train_ds)
    val_acc, val_f1, _, _ = evaluate(model, val_dl)
    print(f"Epoch {epoch:02d} train_loss={train_loss:.4f} val_acc={val_acc:.4f} val_f1={val_f1:.4f}")
    # save best
    if val_f1 > best_val:
        best_val = val_f1
        torch.save(model.state_dict(), OUT_DIR / "best_multimodal_classifier.pth")
        print("Saved best model (f1)", best_val)

Epoch 01 train_loss=1.1833 val_acc=0.9545 val_f1=0.9335
Saved best model (f1) 0.9335073573400268
Epoch 02 train_loss=0.2174 val_acc=1.0000 val_f1=1.0000
Saved best model (f1) 1.0
Epoch 03 train_loss=0.0428 val_acc=1.0000 val_f1=1.0000
Epoch 04 train_loss=0.0176 val_acc=1.0000 val_f1=1.0000
Epoch 05 train_loss=0.0101 val_acc=1.0000 val_f1=1.0000
Epoch 06 train_loss=0.0069 val_acc=1.0000 val_f1=1.0000
Epoch 07 train_loss=0.0051 val_acc=1.0000 val_f1=1.0000
Epoch 08 train_loss=0.0039 val_acc=1.0000 val_f1=1.0000
Epoch 09 train_loss=0.0031 val_acc=1.0000 val_f1=1.0000
Epoch 10 train_loss=0.0026 val_acc=1.0000 val_f1=1.0000
Epoch 11 train_loss=0.0021 val_acc=1.0000 val_f1=1.0000
Epoch 12 train_loss=0.0018 val_acc=1.0000 val_f1=1.0000
Epoch 13 train_loss=0.0016 val_acc=1.0000 val_f1=1.0000
Epoch 14 train_loss=0.0014 val_acc=1.0000 val_f1=1.0000
Epoch 15 train_loss=0.0012 val_acc=1.0000 val_f1=1.0000
Epoch 16 train_loss=0.0011 val_acc=1.0000 val_f1=1.0000
Epoch 17 train_loss=0.0010 val_acc=1.

In [12]:
#  load best model and show report
model.load_state_dict(torch.load(OUT_DIR / "best_multimodal_classifier.pth", map_location=CLASSIFIER_DEVICE))
val_acc, val_f1, preds, truths = evaluate(model, val_dl)
print("Val accuracy:", val_acc, "Val F1:", val_f1)
print("\nClassification report:\n")
print(classification_report(truths, preds, target_names=class_names, digits=4))

# confusion matrix 
import matplotlib.pyplot as plt
import seaborn as sns
cm = confusion_matrix(truths, preds)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt="d", xticklabels=class_names, yticklabels=class_names)
plt.xlabel("Predicted"); plt.ylabel("True")
plt.title("Confusion matrix")
plt.show()

Val accuracy: 1.0 Val F1: 1.0

Classification report:

              precision    recall  f1-score   support

   cardboard     1.0000    1.0000    1.0000        84
       glass     1.0000    1.0000    1.0000        98
       metal     1.0000    1.0000    1.0000        93
       paper     1.0000    1.0000    1.0000       116
     plastic     1.0000    1.0000    1.0000        92
       trash     1.0000    1.0000    1.0000        23

    accuracy                         1.0000       506
   macro avg     1.0000    1.0000    1.0000       506
weighted avg     1.0000    1.0000    1.0000       506



In [13]:
#  inference function (image path + short text)
from PIL import Image

# helper: encode single pair and run classifier
def infer(image_path: str, text: str):
    clip.to(device)  # make sure CLIP on device
    clip.eval()
    proc = processor(images=Image.open(image_path).convert("RGB"), text=text, return_tensors="pt")
    pixel_values = proc["pixel_values"].to(device)
    input_ids = proc["input_ids"].to(device)
    attention_mask = proc["attention_mask"].to(device)
    with torch.no_grad():
        img_emb = clip.get_image_features(pixel_values=pixel_values)
        txt_emb = clip.get_text_features(input_ids=input_ids, attention_mask=attention_mask)
        img_emb = img_emb / img_emb.norm(p=2, dim=-1, keepdim=True)
        txt_emb = txt_emb / txt_emb.norm(p=2, dim=-1, keepdim=True)
        feat = torch.cat([img_emb.cpu(), txt_emb.cpu()], dim=1).to(CLASSIFIER_DEVICE)  # (1, D)
        logits = model(feat)
        probs = torch.softmax(logits, dim=1).cpu().numpy()[0]
        pred_idx = int(torch.argmax(logits, dim=1).cpu().item())
        return {"pred_idx": pred_idx, "pred_class": class_names[pred_idx], "probs": probs}

# Example usage 
example_img = train_list[0][0]  # first training image path
example_class = train_list[0][2]
print("Example true class:", example_class)
res = infer(example_img, f"a photo of a {example_class}")
print(res)

Example true class: glass
{'pred_idx': 1, 'pred_class': 'glass', 'probs': array([0.00278424, 0.9381813 , 0.02822738, 0.00457414, 0.01905029,
       0.0071827 ], dtype=float32)}
